<a href="https://colab.research.google.com/github/lagoda-dmitry/NM/blob/main/Lab3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import plotly.offline as offline
from plotly.graph_objs import *

In [2]:
def get_array_x():
    return [0.0, 1.0, 2.0, 3.0, 4.0]

In [3]:
def get_array_f():
    return [1.0, 1.5403, 1.5839, 2.01, 3.3464]

In [4]:
point_x = 1.5


Построим функцию по точкам


In [5]:
trace = Scatter(
    x = get_array_x(),
    y = get_array_f(),
    name = 'F(x)',
    text = '(x,f(x))',
    showlegend=True
)

data = [trace]

layout = Layout(
    title = 'Cubic interpolation',
    xaxis = dict(title = 'x'),
    yaxis = dict(title = 'y')
)

fig = Figure(data = data, layout = layout)
offline.iplot(fig)


Метод прогонки


In [6]:
def race_algorithm(matrix, d, shape):
    a, b, c = zip(*matrix)
    p = [-c[0] / b[0]]
    q = [d[0] / b[0]]
    x = [0] * (shape + 1)
    for i in range(1, shape):
        p.append(-c[i] / (b[i] + a[i] * p[i - 1]))
        q.append((d[i] - a[i] * q[i - 1]) / (b[i] + a[i] * p[i - 1]))
    for i in reversed(range(shape)):
        x[i] = p[i] * x[i + 1] + q[i]
    return x[:-1]


Поиск коэффициентов


Функции вычисления коэффицентов $ h_i = x_i - x_{i-1} $ и $ c_i $, удовлетворяющих условию: $ h_{i-1} c_{i-1} + 2 (h_{i-1} + h_i) + h_i c_{i+1} = 3 \left(\displaystyle\frac{f_i - f_{i-1}}{h_i} - \displaystyle\frac{f_{i-1} - f_{i-2}}{h_{i-1}}\right) $, для $ i = 2,3,...,n $, где $ c_1 = c_{n+1} = 0 $


In [7]:
def h_coeff(x):
    h = [x[i] - x[i - 1] for i in range(1, len(x))]
    return h

In [8]:
def c_coeff(h, f):
    A = [[h[i - 1], 2.0 * (h[i - 1] + h[i]), h[i]] for i in range(1, len(h))]
    A[0][0] = A[-1][2] = 0.0
    B = [3.0 * ((f[i + 1] - f[i]) / h[i] - (f[i] - f[i - 1]) / h[i - 1]) for i in range(1, len(h))]
    return [0.0] + race_algorithm(A, B, len(h) - 1)



Функции вычисления остальных коэффицентов: $ a_i = f_{i-1} $,

$b_i = \displaystyle\frac{(f_i - f_{i-1})}{h_i} - \displaystyle\frac{h_i}{3}(c_{i+1} + 2c_i)$,

$ d_i = \displaystyle\frac{c_{i+1} - c_i}{3 h_i} $,

для $ i = 1,3,...,n $, где $ c_1 = c_{n+1} = 0 $

In [9]:
def a_coeff(f):
    return f[:len(f) - 1]

In [10]:


def b_coeff(f, h, c):
    b = [(f[i] - f[i - 1]) / h[i - 1] - (1.0/3.0) * h[i - 1] * (c[i] + 2.0 * c[i - 1]) for i in range(1, len(h))]
    b.append((f[-1] - f[-2]) / h[-1] - (2.0/3.0) * h[-1] * c[-1])
    return b



In [11]:
def d_coeff(h, c):
    d = [(c[i + 1] - c[i]) / (3.0 * h[i]) for i in range(len(h) - 1)]
    d.append(-c[-1] / (3.0 * h[-1]))
    return d



Функция вычисляет многочлен кубической интерполяции:

$ S_n(x) = a_i + b_{i}(x - x_{i-1}) + c_{i}(x - x_{i-1})^2 + d_{i}(x - x_{i-1})^3 $,

при $ x_{i-1} \leq x \leq x_i $, $ i = 1, 2, ..., n. $

На каждой из подотрезков кубический сплайн gi(x)определяется как $ S_n(x) = a_i + b_{i}(x - x_{i-1}) + c_{i}(x - x_{i-1})^2 + d_{i}(x - x_{i-1})^3 $, $ i = 1, 2, ..., n - 1. $


In [12]:


def find_position(x, arr):
    for i in range(len(arr) - 1):
        if (arr[i] <= x and x <= arr[i + 1]):
            return i



In [13]:
def interpolation(x):
    X = get_array_x()
    y = get_array_f()
    
    h = h_coeff(X)
    c = c_coeff(h, y)
    a = a_coeff(y)
    b = b_coeff(y, h, c)
    d = d_coeff(h, c)
    
    pos = find_position(x, X)
    
    if pos < 0:
        return b[0]*x + a[0] - b[0]*X[0]
    elif pos == len(X) - 1:
        return Y[-1] + (b[-1] + 2.0*c[-1]*h[-1] + 3.0*d[-1]*h[-1]*h[-1])*(x - X[-1])
    return a[pos] + b[pos]*(x - X[pos]) + c[pos]*((x - X[pos])**2) + d[pos] * ((x - X[pos])**3)


Кубическая интерполяция.

Значение в точке $X^* = 1.5$


In [14]:
Sn = interpolation
Sn(point_x)

1.5862379464285716

In [15]:
x1 = get_array_x()

y1 = list(map(Sn, x1)) 

trace1 = Scatter(
    x = x1,
    y = y1,
    mode = 'markers + lines',
    name = 'Sn(x)',
    text = '(x[i], Sn[i])'
)

trace2 = Scatter(
    x = get_array_x(),
    y = get_array_f(),
    name = 'F(x)',
    mode = 'markers',
    text = '(x[i], f[i])'
)
data = [trace1, trace2]

layout = Layout(
    title = 'Cubic interpolation',
    xaxis = dict(title = 'x'),
    yaxis = dict(title = 'y')
)

fig = Figure(data = data, layout = layout)
offline.iplot(fig)

In [17]:
x_ = np.arange(0, 5.7, 0.005)

y_ = list(map(Sn, x_)) 

trace = Scatter(
    x = x_,
    y = y_,
    name = 'Sn(x)',
    text = '(x[i], y[i])',
    mode = 'markers + lines',
    showlegend=True
)

data = [trace]

layout = Layout(
    title = 'Cubic interpolation',
    xaxis = dict(title = 'x'),
    yaxis = dict(title = 'y')
)

fig = Figure(data = data, layout = layout)
offline.iplot(fig)

TypeError: ignored